In [1]:
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
sys.path.append('../GEECS-PythonAPI')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Undulator")

In [2]:
# set up the python objects for the geecs devices to be used in optimization
hexapod = GeecsDevice('U_Hexapod')

# define a dict containing the specific variables and bounds to be used in optimization
objs={'HexY':{"GEECS_Object":hexapod,'variable':"ypos","bounds":[17,19]},
    'HexZ':{"GEECS_Object":hexapod,'variable':"zpos","bounds":[-.4,1]},
      'HexV':{"GEECS_Object":hexapod,'variable':"vangle","bounds":[-.4,.1]},
     'HexW':{"GEECS_Object":hexapod,'variable':"wangle","bounds":[-.4,.1]}}

In [22]:
#define whether or not to use normalized values and ranges for optimization controls. 
#Normalization seems like a good idea
normalize=False

#for opt method, choose nelder or bayes for now
opt_method='bayes'

#define the xopt configuration
YAML = """
xopt:
    dump_file: dump.yaml
generator:
    name:
evaluator:
    function: __main__.geecs_measurement

vocs:
    variables:
        {}
    objectives: {f: "MAXIMIZE"}

"""

yaml_output = yaml.safe_load(YAML)

if opt_method == 'bayes':
    yaml_output['generator']['name'] = 'upper_confidence_bound'
    yaml_output['generator']['n_initial'] = 2
    # yaml_output['generator']['acq'] = {'beta':0.1}
    yaml_output['xopt']['dump_file'] = 'bayes.yaml'
elif opt_method == 'nelder':
    yaml_output['generator']['name'] = 'neldermead'
    yaml_output['generator']['adaptive'] = True
    yaml_output['generator']['xatol'] = 0.01
    yaml_output['generator']['fatol'] = 0.005
    yaml_output['generator']['initial_point'] = { "HexY":18.0,"HexZ":0.1,"HexV":0,"HexW":0}
    yaml_output['xopt']['dump_file'] = 'nelder.yaml'

#define some functions to normalize/unnormalize real variables to [-1,1]
def unnormalize_var(obj,val):
    range=objs[obj]['bounds'][1]-objs[obj]['bounds'][0]
    offset=objs[obj]['bounds'][0]
    new_val=(val/2+0.5)*range+offset
    return new_val

def normalize_var(obj,val):
    range=objs[obj]['bounds'][1]-objs[obj]['bounds'][0]
    offset=objs[obj]['bounds'][0]
    new_val=((val - offset)/range - 0.5)*2
    return new_val
    

for tag in objs.keys():
    yaml_output['vocs']['variables'][tag]=objs[tag]['bounds']
    
if normalize:
    for tag in objs.keys():
        yaml_output['vocs']['variables'][tag]=[-1.0,1.0]
    keys = yaml_output['vocs']['variables'].keys()
    

if yaml_output['generator']['name']=='neldermead':
    if normalize:
        initial_point = yaml_output['generator']['initial_point']

        normalized_initial_point = {}
        for key in keys:
            normalized_initial_point[key] = normalize_var(key, initial_point[key])

        yaml_output['generator']['initial_point'] = normalized_initial_point

print(yaml_output)

{'xopt': {'dump_file': 'bayes.yaml'}, 'generator': {'name': 'upper_confidence_bound', 'n_initial': 2}, 'evaluator': {'function': '__main__.geecs_measurement'}, 'vocs': {'variables': {'HexY': [17, 19], 'HexZ': [-0.4, 1], 'HexV': [-0.4, 0.1], 'HexW': [-0.4, 0.1]}, 'objectives': {'f': 'MAXIMIZE'}}}


In [23]:
#simulate transmission of blue diode through PMQ triplet
import numpy as np

optPosition = np.array([18.45, 0.6])
numParts = 200000

startDist = np.transpose([
    np.random.normal(optPosition[0], 0.4, numParts),
    np.random.normal(optPosition[1], 0.4, numParts)
])

def calcTransmission(input_dict):
    center1 = [input_dict['HexY'], input_dict['HexZ']]
    separation = 15

    center2 = [input_dict['HexY'], input_dict['HexZ']]
    rotw = np.pi / 180 * (input_dict['HexW'] + 0.15) * 4
    rotv = np.pi / 180 * (input_dict['HexV'] + 0.25) * 4

    yOffset = separation * np.tan(rotw)
    zOffset = separation * np.tan(rotv)

    center2[0] = center2[0] + yOffset
    center2[1] = center2[1] + zOffset

    dist = startDist[
        (np.sqrt((startDist[:, 0] - center1[0])**2 + (startDist[:, 1] - center1[1])**2) < 0.2) &
        (np.sqrt((startDist[:, 0] - center2[0])**2 + (startDist[:, 1] - center2[1])**2) < 0.2)
    ]

    return len(dist) / numParts

In [10]:
def geecs_measurement(input_dict):
    for i in list(input_dict.keys()):
        set_val=float(input_dict[i])
        if normalize:
            set_val=unnormalize_var(i,set_val)
        
        print('set '+str(i)+' to '+str(set_val))
        
        #line below executes the set commands.
        #objs[i]["GEECS_Object"].set(objs[i]["variable"],set_val)
        time.sleep(0)
        
    if normalize:
        setpoint = {}
        for key in input_dict:
            setpoint[key] = unnormalize_var(key, input_dict[key])
    else:
        setpoint=input_dict
        
    print(setpoint)
    value=calcTransmission(setpoint)

    return {'f': value}


In [11]:
# if opt_method == 'nelder':
#     geecs_measurement(yaml_output['generator']['initial_point'])
    
# geecs_measurement(yaml_output['generator']['initial_point'])


In [24]:
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import ExpectedImprovementGenerator
from xopt import Xopt

X = Xopt(config=yaml_output)
X


            Xopt
________________________________
Version: 1.4.1
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: false, dump_file: bayes.yaml, max_evaluations: null}
generator:
  name: upper_confidence_bound
  optim: {num_restarts: 20, raw_samples: 20, sequential: true, max_travel_distances: null,
    use_turbo: false}
  acq: {proximal_lengthscales: null, use_transformed_proximal_weights: true, monte_carlo_samples: 128,
    beta: 2.0}
  model:
    name: standard
    custom_constructor: null
    use_low_noise_prior: true
    covar_modules: {}
    mean_modules: {}
  n_initial: 2
  use_cuda: false
evaluator:
  function: __main__.geecs_measurement
  max_workers: 1
  function_kwargs: {}
  vectorized: false
vocs:
  variables:
    HexY: [17.0, 19.0]
    HexZ: [-0.4, 1.0]
    HexV: [-0.4, 0.1]
    HexW: [-0.4, 0.1]
  constraints: {}
  objectives: {f: MAXIMIZE}
  constants: {}
  linked_variables: {}


In [13]:
if opt_method == 'bayes':
    # print initial number of points to be generated
    print(X.generator.options.n_initial)

    # call X.step() to generate + evaluate initial points
    X.step()

    # inspect the gathered data
    X.data

import torch
import matplotlib.pyplot as plt
n_steps = 50

for i in range(n_steps):
    print('step'+str(i))
    X.step()
    


step0
set HexV to 0.0
set HexW to 0.0
set HexY to 18.0
set HexZ to 0.1
{'HexV': 0.0, 'HexW': 0.0, 'HexY': 18.0, 'HexZ': 0.1}
step1
set HexV to 0.00025
set HexW to 0.0
set HexY to 18.0
set HexZ to 0.1
{'HexV': 0.00025, 'HexW': 0.0, 'HexY': 18.0, 'HexZ': 0.1}
step2
set HexV to 0.0
set HexW to 0.00025
set HexY to 18.0
set HexZ to 0.1
{'HexV': 0.0, 'HexW': 0.00025, 'HexY': 18.0, 'HexZ': 0.1}
step3
set HexV to 0.0
set HexW to 0.0
set HexY to 18.900000000000002
set HexZ to 0.1
{'HexV': 0.0, 'HexW': 0.0, 'HexY': 18.900000000000002, 'HexZ': 0.1}
step4
set HexV to 0.0
set HexW to 0.0
set HexY to 18.0
set HexZ to 0.10500000000000001
{'HexV': 0.0, 'HexW': 0.0, 'HexY': 18.0, 'HexZ': 0.10500000000000001}
step5
set HexV to 0.000125
set HexW to 0.000125
set HexY to 17.099999999999998
set HexZ to 0.10250000000000001
{'HexV': 0.000125, 'HexW': 0.000125, 'HexY': 17.099999999999998, 'HexZ': 0.10250000000000001}
step6
set HexV to 2.34375e-05
set HexW to 2.34375e-05
set HexY to 18.5625
set HexZ to 0.100468

In [14]:
type(X.data)
df=X.data
last_row_series = df.iloc[-1]
control_values = last_row_series[yaml_output['vocs']['variables'].keys()]  # control_names is a list of your control column names
f_value = last_row_series['f']
last_row_dict = df.iloc[-1].to_dict()
last_row_dict
X.data
x = df['f']
x.values

array([6.9900e-03, 6.9750e-03, 6.9750e-03, 4.4750e-03, 7.0950e-03,
       5.0000e-05, 9.6450e-03, 1.0535e-02, 1.1015e-02, 1.1160e-02,
       9.5700e-03, 7.8550e-03, 3.1700e-03, 9.7450e-03, 8.1950e-03,
       9.8550e-03, 7.6000e-03, 1.1115e-02, 9.8900e-03, 7.4950e-03,
       1.0695e-02, 1.0380e-02, 1.0670e-02, 1.0760e-02, 9.3450e-03,
       1.1010e-02, 1.0865e-02, 1.0950e-02, 1.1105e-02, 1.0475e-02,
       1.0955e-02, 1.1115e-02, 1.1065e-02, 1.0995e-02, 1.0930e-02,
       1.0740e-02, 1.1085e-02, 1.0555e-02, 1.1125e-02, 1.1130e-02,
       1.1200e-02, 1.1075e-02, 1.1200e-02, 1.1205e-02, 1.1110e-02,
       1.0890e-02, 1.0945e-02, 1.1200e-02, 1.1045e-02, 1.1170e-02])

In [ ]:
import matplotlib.pyplot as plt
X.data['f'].plot(marker='.')
plt.yscale('log')
plt.xlabel('iteration')
plt.ylabel('Rosenbrock value')
best=dict(X.data.iloc[X.data["f"].argmax()]);

if normalize:
    for key in yaml_output['vocs']['variables'].keys():
        best[key] = unnormalize_var(key, best[key])

best

In [ ]:
X.data.empty